# Project 1B: Data Modeling with Apache Cassandra

## Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
         # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Create Keyspace and tables in Apache Cassandra

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Connect to a Cassandra instance on your local machine (127.0.0.1)
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute(
"""
CREATE KEYSPACE IF NOT EXISTS sparkify
WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("sparkify")
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

1. Return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Return name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Define common functions

### Create table function

In [8]:
def create_table_query(table_name, columns_types):
    query = f"CREATE TABLE IF NOT EXISTS {table_name}"
    query += f"({columns_types})"
    try:
        session.execute(query)
    except Exception as e:
        print(e)

### Insert data into table function

In [9]:
file = 'event_datafile_new.csv'
def insert_data_in_table(table_name, cols_info):
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = f"INSERT INTO {table_name} (" + ",".join([a for a, _, _ in cols_info]) + ")"
            query += "VALUES (" + ",".join(["%s"] * len(cols_info)) + ")"
            row_info = [v(line[k]) for _, k, v in cols_info]
            session.execute(query, row_info)

### Drop table function

In [10]:
def drop_table(table_name):
    try:
        session.execute(f"drop table if exists {table_name}")
    except Exception as e:
        print(e)

## Task 1: Create and populate sessions_history table
- Requirement: Return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
- Query: `SELECT artist, song_title, length from sessions_history where session_id=338 and item_in_session=4;`
- Resulting table: sessions_history

In [11]:
# Create table
session_history_cols = """
session_id int, 
item_in_session int, 
artist text, 
song_title text, 
length decimal, 
primary key (session_id, item_in_session)
"""
create_table_query("sessions_history", session_history_cols)

In [12]:
# Insert data in table
sessions_history_info = [
    ("session_id", 8, int),
    ("item_in_session", 3, int),
    ("artist", 0, str),
    ("song_title", 9, str),
    ("length", 5, float),
]
insert_data_in_table("sessions_history", sessions_history_info)

In [13]:
# Verify the data was entered into the table
rows = session.execute("SELECT artist, song_title, length from sessions_history where session_id=338 and item_in_session=4")
for row in rows:
    print(row)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', length=Decimal('495.3073'))


## Task 2: Create and populate users_sessions table
- Requirement: Return name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- Query: `SELECT artist, song_title, user_first_name, user_last_name from users_sessions where user_id=10 and session_id=182;`
- Resulting table: users_sessions

In [14]:
# Create table
users_sessions_cols = """
user_id int, 
session_id int,
item_in_session int,
artist text, 
song_title text, 
user_first_name text,
user_last_name text, 
primary key ((user_id, session_id), item_in_session)
"""
create_table_query("users_sessions", users_sessions_cols)

In [15]:
# Insert data into table
users_sessions_info = [
    ("user_id", 10, int),
    ("session_id", 8, int),
    ("item_in_session", 3, int),
    ("artist", 0, str),
    ("song_title", 9, str),
    ("user_first_name", 1, str),
    ("user_last_name", 4, str),
]
insert_data_in_table("users_sessions", users_sessions_info)

In [16]:
# Verify the data was entered into the table
rows = session.execute("SELECT artist, song_title, user_first_name, user_last_name from users_sessions where user_id=10 and session_id=182")
for row in rows:
    print(row)

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')
Row(artist='Three Drives', song_title='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')


## Create and populate users_per_song table (query 3)
- Requirement: Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
- Query: `SELECT user_first_name, user_last_name from users_per_song where song_title='All Hands Against His Own';`
- Resulting table: users_per_song

In [17]:
# Create table
users_per_song_cols = """
song_title text,
user_id int,
user_first_name text, 
user_last_name text, 
primary key (song_title, user_id)
"""
create_table_query("users_per_song", users_per_song_cols)

In [18]:
# Insert data into table
users_per_song_info = [
    ("song_title", 9, str),
    ("user_id", 10, int),
    ("user_first_name", 1, str),
    ("user_last_name", 4, str),
]
insert_data_in_table("users_per_song", users_per_song_info)

In [19]:
# Verify the data was entered into the table
rows = session.execute("SELECT user_first_name, user_last_name from users_per_song where song_title='All Hands Against His Own'")
for row in rows:
    print(row)

Row(user_first_name='Jacqueline', user_last_name='Lynch')
Row(user_first_name='Tegan', user_last_name='Levine')
Row(user_first_name='Sara', user_last_name='Johnson')


### Drop the tables before closing out the sessions

In [20]:
# Drop the table before closing out the sessions
drop_table("sessions_history")
drop_table("users_sessions")
drop_table("users_per_song")

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()